In [1]:
#ライブラリのimport
import pandas as pd
import PIL.Image
import PIL.ImageTk
import PIL.ImageDraw
import PIL.ImageFont
import tkinter as tk
import numpy as np

In [2]:
#データセットの読み込み
lett_data = pd.read_csv("文字素材.csv",header=None)
#末尾に" "(空白)を追加
lett_data.loc[len(lett_data)] = [" "]
lett_data

,0
0,a
1,b
2,c
3,d
4,e
...,...
88,/
89,.
90,","
91,~


In [3]:
#文字から明るさを決定する関数
def determining_brightness(lett):
    #画像を準備
    img = PIL.Image.new("L",(150,300),color=255)
    draw = PIL.ImageDraw.Draw(img)

    #フォント情報を作成
    font_size = 10
    font = PIL.ImageFont.truetype("MSMINCHO.TTF",300)

    #フォントを書く
    draw.text((0,0),lett,font=font)

    gray_image = img.convert("L")
    gray_image = gray_image.resize((1,1),PIL.Image.Resampling.LANCZOS)

    #明るさを取り出す
    blight_data = np.asarray(gray_image)

    #そのままだとリストなので，数字のみを取り出す
    blightness = blight_data[0][0]

    return blightness

In [4]:
#明るさを0~1にする関数
def normalization_to_0_1(num):
    num = float(abs(num-255)/255)
    return num

In [5]:
#最大値を天井とし，0~1で正規化
def normalization_to_0_1_EX(num,max,min):
    num = float(num/(max-min))
    return num

#### データフレームを作成する  
A列→index  
B列→文字  
C列→明るさ(255~0)  
D列→明るさ(0~1正規化)  
E列→明るさ(0~1max,min正規化)

In [6]:
#データフレームを作成
df = pd.DataFrame()
#文字データをコピーして0列目に格納
df[0] = lett_data[0]

#明るさを計算して1列目に格納
df[1] = df[0].apply(determining_brightness)
#正規化したデータを2列目に格納
df[2] = df[1].apply(normalization_to_0_1)
#最大値，最小値を求める
max = df[2].max()
min = df[2].min()
#最大値を1としてデータを3列目に格納
df[3] = df[2].apply(lambda row : normalization_to_0_1_EX(row,max,min))

#値が大きい順に並べ替える
sorted_df = df.sort_values(by=3,ascending=False)
#何故か並び替えができないので，データセット取得時に行う
""" #並べ替えたものをdfに格納(文字とmax,min正規化の数値のみ)
df[5] = sorted_df[0]
df[6] = sorted_df[3] """


#csvに保存する
df.to_csv("DataSets_MSMincho.csv",mode="x",header=False)

#sorted_df